In [1]:
import os
import sys
import glob
import subprocess
import shutil
import time
from pathlib import Path
from functools import lru_cache
import torch
import fla.utils

In [ ]:
# ==============================================================================
# 2. MODEL LOADING (PAPER REPLICATION MODE)
# ==============================================================================
import torch
from pathlib import Path
from peft import PeftModel
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from fla.models.delta_net import DeltaNetForCausalLM, DeltaNetConfig

# --- Configuration ---
USE_ADAPTER = False 
BASE_MODEL_ID = "fla-hub/delta_net-1.3B-100B"
DEVICE_MAP = "cuda:0" # Explicitly set to GPU

# --- REPLICATION FACTOR 1: PRECISION ---
# The paper used bfloat16 (standard for 1.3B training).
# We stick to this. (Unlike our previous 'float32' fix which boosted scores).
DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

# --- A. Register Architecture ---
try:
    AutoConfig.register("delta_net", DeltaNetConfig)
    AutoModelForCausalLM.register(DeltaNetConfig, DeltaNetForCausalLM)
    print("✅ DeltaNet architecture registered.")
except ValueError:
    pass

# --- B. Load Base Model ---
print(f"⏳ Loading Base Model: {BASE_MODEL_ID}...")
model = DeltaNetForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    trust_remote_code=True,
    device_map=DEVICE_MAP,
    torch_dtype=DTYPE,
)

# --- C. Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # LM-Harness usually prefers left for gen, but right is standard for base models

# --- D. REPLICATION FACTOR 2: Context Limit (CRITICAL) ---
# Your previous code: 131072 (128k) -> Allowed extrapolation
# Paper Replication:  4096  (4k)   -> Forces truncation on long tasks
#model.config.max_position_embeddings = 4096
#model.config.max_length = 4096
#tokenizer.model_max_length = 4096

model.eval()

`torch_dtype` is deprecated! Use `dtype` instead!


✅ DeltaNet architecture registered.
⏳ Loading Base Model: fla-hub/delta_net-1.3B-100B...


DeltaNetForCausalLM(
  (model): DeltaNetModel(
    (embeddings): Embedding(32000, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-23): 24 x DeltaNetBlock(
        (attn_norm): RMSNorm(2048, eps=1e-06)
        (attn): DeltaNet(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (b_proj): Linear(in_features=2048, out_features=16, bias=False)
          (q_conv1d): ShortConvolution(2048, 2048, kernel_size=(4,), stride=(1,), padding=(3,), groups=2048, bias=False, activation=silu, backend=triton)
          (k_conv1d): ShortConvolution(2048, 2048, kernel_size=(4,), stride=(1,), padding=(3,), groups=2048, bias=False, activation=silu, backend=triton)
          (v_conv1d): ShortConvolution(2048, 2048, kernel_size=(4,), stride=(1,), padding=(3,), groups=2048, bias=False, activation=silu, backend=triton)
 

In [4]:
from lm_eval import evaluator
results = evaluator.simple_evaluate(
    model="hf",
    model_args=(
        "pretrained=fla-hub/delta_net-1.3B-100B,"
        "tokenizer=fla-hub/delta_net-1.3B-100B,"
        "trust_remote_code=true,"
        "max_length=16384,"
    ),
    tasks=["niah_single_1", "niah_single_2", "niah_single_3"],
    batch_size=1,
    metadata={
        "max_seq_lengths": [1024, 2048, 4096, 8192],
        "tokenizer": "fla-hub/delta_net-1.3B-100B"
    }
)

niah_single_1: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: repeat | 1024: 100%|██████████████████████████████████| 500/500 [00:00<00:00, 1553.20it/s]
Generating synthetic samples: repeat | 2048: 100%|███████████████████████████████████| 500/500 [00:00<00:00, 649.14it/s]
Generating synthetic samples: repeat | 4096: 100%|███████████████████████████████████| 500/500 [00:01<00:00, 333.85it/s]
Generating synthetic samples: repeat | 8192: 100%|███████████████████████████████████| 500/500 [00:03<00:00, 160.64it/s]
niah_single_2: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 1024: 100%|███████████████████████████████████| 500/500 [00:00<00:00, 1328.01it/s]

In [6]:
from lm_eval.utils import make_table

# Print formatted table
print(make_table(results))

# Or print just the scores
if "results" in results:
    for task_name, task_results in results["results"].items():
        print(f"\n{task_name}:")
        for metric, value in task_results.items():
            print(f"  {metric}: {value}")

|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_1|      1|none  |     0|  1024|   |1.000|±  |     0|
|             |       |none  |     0|  2048|   |1.000|±  |     0|
|             |       |none  |     0|  4096|↑  |1.000|±  |   N/A|
|             |       |none  |     0|  8192|↑  |1.000|±  |   N/A|
|niah_single_2|      1|none  |     0|  1024|   |1.000|±  |     0|
|             |       |none  |     0|  2048|   |1.000|±  |     0|
|             |       |none  |     0|  4096|↑  |0.666|±  |   N/A|
|             |       |none  |     0|  8192|↑  |0.248|±  |   N/A|
|niah_single_3|      1|none  |     0|  1024|   |0.874|±  |0.0149|
|             |       |none  |     0|  2048|   |0.730|±  |0.0199|
|             |       |none  |     0|  4096|↑  |0.548|±  |   N/A|
|             |       |none  |     0|  8192|↑  |0.096|±  |   N/A|


niah_single_1:
  4096,none: 1.0
  4096_stderr,none: N/A
  1024,none: 1.0
 

In [ ]:
# ==============================================================================
# 4. RULER EVALUATION HARNESS 
# ==============================================================================
import lm_eval
from lm_eval.models.huggingface import HFLM
from lm_eval import simple_evaluate
from lm_eval.utils import make_table
import json

print("\n🔌 Plugging DeltaNet into Evaluation Harness...")

# 2. WRAP THE MODEL
lm_obj = HFLM(
    pretrained=model,
    tokenizer=tokenizer,
    batch_size=1,
)
# ---------------------------------------

# 3. DEFINE TASK
TASK_NAME = "niah_single_2" 
LENGTHS_TO_TEST = [2048] 

print(f"🚀 Starting RULER Evaluation: {TASK_NAME}")
print(f"📏 Testing Lengths: {LENGTHS_TO_TEST}")

# 4. RUN EVALUATION
results = simple_evaluate(
    model=lm_obj,
    tasks=[TASK_NAME],
    device="cuda",
    limit=50,
    metadata={
        "max_seq_lengths": LENGTHS_TO_TEST,
        "tokenizer": BASE_MODEL_ID 
    }
)

# 5. PRINT & SAVE
print("\n" + "="*40)
print(f"🏆 RESULTS: DeltaNet ({'Fine-Tuned' if USE_ADAPTER else 'Baseline'})")
print("="*40)
print(make_table(results))

output_file = f"results_deltanet_{'finetuned' if USE_ADAPTER else 'baseline'}.json"
with open(output_file, "w") as f:
    json.dump(results["results"], f, indent=4)
print(f"\n💾 Results saved to {output_file}")

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
HF model type is neither marked as CausalLM or Seq2SeqLM.                     This is expected if your model requires `trust_remote_code=True` but may be an error otherwise.Setting backend to causal
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration



🔌 Plugging DeltaNet into Evaluation Harness...
🚀 Starting RULER Evaluation: niah_single_2
📏 Testing Lengths: [2048]


niah_single_2: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 2048: 100%|████████████████████████████████████| 500/500 [00:00<00:00, 565.77it/s]
Running generate_until requests: 100%|██████████████████████████████████████████████████| 50/50 [07:00<00:00,  8.42s/it]



🏆 RESULTS: DeltaNet (Baseline)
|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_2|      1|none  |     0|  2048|   |    1|±  |     0|
|             |       |none  |     0|  4096|↑  |   -1|±  |   N/A|


💾 Results saved to results_deltanet_baseline.json


In [ ]:
# ==============================================================================
# 4. RULER EVALUATION HARNESS 
# ==============================================================================
import lm_eval
from lm_eval.models.huggingface import HFLM
from lm_eval import simple_evaluate
from lm_eval.utils import make_table
import json

print("\n🔌 Plugging DeltaNet into Evaluation Harness...")

# 2. WRAP THE MODEL
lm_obj = HFLM(
    pretrained=model,
    tokenizer=tokenizer,
    batch_size=1,
)
# ---------------------------------------

# 3. DEFINE TASK
TASK_NAME = "niah_single_3" 
LENGTHS_TO_TEST = [2048] 

print(f"🚀 Starting RULER Evaluation: {TASK_NAME}")
print(f"📏 Testing Lengths: {LENGTHS_TO_TEST}")

# 4. RUN EVALUATION
results = simple_evaluate(
    model=lm_obj,
    tasks=[TASK_NAME],
    device="cuda",
    limit=50,
    metadata={
        "max_seq_lengths": LENGTHS_TO_TEST,
        "tokenizer": BASE_MODEL_ID 
    }
)

# 5. PRINT & SAVE
print("\n" + "="*40)
print(f"🏆 RESULTS: DeltaNet ({'Fine-Tuned' if USE_ADAPTER else 'Baseline'})")
print("="*40)
print(make_table(results))

output_file = f"results_deltanet_{'finetuned' if USE_ADAPTER else 'baseline'}.json"
with open(output_file, "w") as f:
    json.dump(results["results"], f, indent=4)
print(f"\n💾 Results saved to {output_file}")

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
HF model type is neither marked as CausalLM or Seq2SeqLM.                     This is expected if your model requires `trust_remote_code=True` but may be an error otherwise.Setting backend to causal
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration



🔌 Plugging DeltaNet into Evaluation Harness...
🚀 Starting RULER Evaluation: niah_single_3
📏 Testing Lengths: [2048]


niah_single_3: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 2048: 100%|████████████████████████████████████| 500/500 [00:01<00:00, 262.23it/s]
Running generate_until requests: 100%|██████████████████████████████████████████████████| 50/50 [06:44<00:00,  8.09s/it]



🏆 RESULTS: DeltaNet (Baseline)
|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_3|      1|none  |     0|  2048|   | 0.72|±  |0.0641|
|             |       |none  |     0|  4096|↑  |-1.00|±  |   N/A|


💾 Results saved to results_deltanet_baseline.json


In [ ]:
# ==============================================================================
# 4. RULER EVALUATION HARNESS 
# ==============================================================================
import lm_eval
from lm_eval.models.huggingface import HFLM
from lm_eval import simple_evaluate
from lm_eval.utils import make_table
import json

print("\n🔌 Plugging DeltaNet into Evaluation Harness...")

# 2. WRAP THE MODEL
lm_obj = HFLM(
    pretrained=model,
    tokenizer=tokenizer,
    batch_size=1,
)
# ---------------------------------------

# 3. DEFINE TASK
TASK_NAME = "niah_single_2" 
LENGTHS_TO_TEST = [2048] 

print(f"🚀 Starting RULER Evaluation: {TASK_NAME}")
print(f"📏 Testing Lengths: {LENGTHS_TO_TEST}")

# 4. RUN EVALUATION
results = simple_evaluate(
    model=lm_obj,
    tasks=[TASK_NAME],
    device="cuda",
    limit=20,
    metadata={
        "max_seq_lengths": LENGTHS_TO_TEST,
        "tokenizer": BASE_MODEL_ID 
    }
)

# 5. PRINT & SAVE
print("\n" + "="*40)
print(f"🏆 RESULTS: DeltaNet ({'Fine-Tuned' if USE_ADAPTER else 'Baseline'})")
print("="*40)
print(make_table(results))

output_file = f"results_deltanet_{'finetuned' if USE_ADAPTER else 'baseline'}.json"
with open(output_file, "w") as f:
    json.dump(results["results"], f, indent=4)
print(f"\n💾 Results saved to {output_file}")

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
HF model type is neither marked as CausalLM or Seq2SeqLM.                     This is expected if your model requires `trust_remote_code=True` but may be an error otherwise.Setting backend to causal
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration



🔌 Plugging DeltaNet into Evaluation Harness...
🚀 Starting RULER Evaluation: niah_single_2
📏 Testing Lengths: [2048]


niah_single_2: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 2048: 100%|████████████████████████████████████| 500/500 [00:00<00:00, 649.87it/s]
Running generate_until requests:   0%|                                                           | 0/20 [00:00<?, ?it/s]/home/louis/miniconda3/envs/babilong/lib/python3.11/site-packages/transformers/generation/utils.py:1733: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Running generate_until requests: 100%|██████████████████████████████████████████████████| 20/20 [02:31<00:


🏆 RESULTS: DeltaNet (Baseline)
|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_2|      1|none  |     0|  2048|   |    1|±  |     0|
|             |       |none  |     0|  4096|↑  |   -1|±  |   N/A|


💾 Results saved to results_deltanet_baseline.json


In [19]:
# ==============================================================================
# 4. RULER EVALUATION 2
# ==============================================================================
import lm_eval
from lm_eval.models.huggingface import HFLM
from lm_eval import simple_evaluate
from lm_eval.utils import make_table
import json

print("\n🔌 Plugging DeltaNet into Evaluation Harness...")

# 1. PRE-PATCH THE CONFIG (Just in case)

# 2. WRAP THE MODEL
lm_obj = HFLM(
    pretrained=model,
    tokenizer=tokenizer,
    batch_size=1,
)

# --- 🛠️ STEP 4.5: THE NUCLEAR FIX 🛠️ ---
lm_obj._max_length = 131072
print(f"✅ Force-set Harness Max Length to: {lm_obj.max_length}")
# ---------------------------------------

# 3. DEFINE TASK
TASK_NAME = "niah_single_1" 
LENGTHS_TO_TEST = [2048, 4096, 8192]
#[2048, 4096, 8192] 

print(f"🚀 Starting RULER Evaluation: {TASK_NAME}")
print(f"📏 Testing Lengths: {LENGTHS_TO_TEST}")

# 4. RUN EVALUATION
results = simple_evaluate(
    model=lm_obj,
    tasks=[TASK_NAME],
    device="cuda",
    num_fewshot=2,
    metadata={
        "max_seq_lengths": LENGTHS_TO_TEST,
        "tokenizer": BASE_MODEL_ID 
    }
)

# 5. PRINT & SAVE
print("\n" + "="*40)
print(f"🏆 RESULTS: DeltaNet ({'Fine-Tuned' if USE_ADAPTER else 'Baseline'})")
print("="*40)
print(make_table(results))

output_file = f"results_deltanet_{'finetuned' if USE_ADAPTER else 'baseline'}.json"
with open(output_file, "w") as f:
    json.dump(results["results"], f, indent=4)
print(f"\n💾 Results saved to {output_file}")

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
HF model type is neither marked as CausalLM or Seq2SeqLM.                     This is expected if your model requires `trust_remote_code=True` but may be an error otherwise.Setting backend to causal
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration



🔌 Plugging DeltaNet into Evaluation Harness...
✅ Force-set Harness Max Length to: 131072
🚀 Starting RULER Evaluation: niah_single_1
📏 Testing Lengths: [2048, 4096, 8192]


niah_single_1: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.


Generating synthetic samples: repeat | 2048:   0%|                                              | 0/500 [00:00<?, ?it/s]

Generating synthetic samples: repeat | 2048:  11%|███▉                                | 54/500 [00:00<00:00, 532.98it/s]

Generating synthetic samples: repeat | 2048:  22%|███████▌                           | 108/500 [00:00<00:00, 524.32it/s]

Generating synthetic samples: repeat | 2048:  32%|███████████▎                       | 161/500 [00:00<00:00, 510.11it/s]

Generating synthetic samples: repeat | 2048:  43%|██████████████▉                    | 213/500 [00:00<00:00, 511.61it/s]

Generating synthetic samples: repeat | 2048:  53%|██████████████████▌                | 265/500 [00:00<00:00, 507.99it/s]

Generating synthetic samples: repeat | 2048:  63%|██████████████████


🏆 RESULTS: DeltaNet (Baseline)
|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_1|      1|none  |     2|  2048|   |0.750|±  |0.0194|
|             |       |none  |     2|  4096|↑  |0.656|±  |   N/A|
|             |       |none  |     2|  8192|↑  |0.576|±  |   N/A|


💾 Results saved to results_deltanet_baseline.json


In [20]:
# ==============================================================================
# 4. RULER EVALUATION 2
# ==============================================================================
import lm_eval
from lm_eval.models.huggingface import HFLM
from lm_eval import simple_evaluate
from lm_eval.utils import make_table
import json

print("\n🔌 Plugging DeltaNet into Evaluation Harness...")

# 1. PRE-PATCH THE CONFIG (Just in case)

# 2. WRAP THE MODEL
lm_obj = HFLM(
    pretrained=model,
    tokenizer=tokenizer,
    batch_size=1,
)

# --- 🛠️ STEP 4.5: THE NUCLEAR FIX 🛠️ ---
lm_obj._max_length = 131072
print(f"✅ Force-set Harness Max Length to: {lm_obj.max_length}")
# ---------------------------------------

# 3. DEFINE TASK
TASK_NAME = "niah_single_2" 
LENGTHS_TO_TEST = [2048, 4096, 8192]
#[2048, 4096, 8192] 

print(f"🚀 Starting RULER Evaluation: {TASK_NAME}")
print(f"📏 Testing Lengths: {LENGTHS_TO_TEST}")

# 4. RUN EVALUATION
results = simple_evaluate(
    model=lm_obj,
    tasks=[TASK_NAME],
    device="cuda",
    num_fewshot=2,
    metadata={
        "max_seq_lengths": LENGTHS_TO_TEST,
        "tokenizer": BASE_MODEL_ID 
    }
)

# 5. PRINT & SAVE
print("\n" + "="*40)
print(f"🏆 RESULTS: DeltaNet ({'Fine-Tuned' if USE_ADAPTER else 'Baseline'})")
print("="*40)
print(make_table(results))

output_file = f"results_deltanet_{'finetuned' if USE_ADAPTER else 'baseline'}.json"
with open(output_file, "w") as f:
    json.dump(results["results"], f, indent=4)
print(f"\n💾 Results saved to {output_file}")

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
HF model type is neither marked as CausalLM or Seq2SeqLM.                     This is expected if your model requires `trust_remote_code=True` but may be an error otherwise.Setting backend to causal
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration



🔌 Plugging DeltaNet into Evaluation Harness...
✅ Force-set Harness Max Length to: 131072
🚀 Starting RULER Evaluation: niah_single_2
📏 Testing Lengths: [2048, 4096, 8192]


niah_single_2: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 2048: 100%|██████████████████████████| 500/500 [00:00<00:00, 515.50it/s]
Generating synthetic samples: essay | 4096: 100%|██████████████████████████| 500/500 [00:02<00:00, 190.60it/s]
Generating synthetic samples: essay | 8192: 100%|███████████████████████████| 500/500 [00:06<00:00, 73.87it/s]
Overwriting default num_fewshot of niah_single_2 from None to 2
Running generate_until requests: 100%|██████████████████████████████████| 1500/1500 [3:05:28<00:00,  7.42s/it]



🏆 RESULTS: DeltaNet (Baseline)
|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_2|      1|none  |     2|  2048|   |0.338|±  |0.0212|
|             |       |none  |     2|  4096|↑  |0.306|±  |   N/A|
|             |       |none  |     2|  8192|↑  |0.088|±  |   N/A|


💾 Results saved to results_deltanet_baseline.json


In [21]:
# ==============================================================================
# 4. RULER EVALUATION 2
# ==============================================================================
import lm_eval
from lm_eval.models.huggingface import HFLM
from lm_eval import simple_evaluate
from lm_eval.utils import make_table
import json

print("\n🔌 Plugging DeltaNet into Evaluation Harness...")

# 1. PRE-PATCH THE CONFIG (Just in case)

# 2. WRAP THE MODEL
lm_obj = HFLM(
    pretrained=model,
    tokenizer=tokenizer,
    batch_size=1,
)

# --- 🛠️ STEP 4.5: THE NUCLEAR FIX 🛠️ ---
lm_obj._max_length = 131072
print(f"✅ Force-set Harness Max Length to: {lm_obj.max_length}")
# ---------------------------------------

# 3. DEFINE TASK
TASK_NAME = "niah_single_3" 
LENGTHS_TO_TEST = [2048, 4096, 8192]
#[2048, 4096, 8192] 

print(f"🚀 Starting RULER Evaluation: {TASK_NAME}")
print(f"📏 Testing Lengths: {LENGTHS_TO_TEST}")

# 4. RUN EVALUATION
results = simple_evaluate(
    model=lm_obj,
    tasks=[TASK_NAME],
    device="cuda",
    num_fewshot=2,
    metadata={
        "max_seq_lengths": LENGTHS_TO_TEST,
        "tokenizer": BASE_MODEL_ID 
    }
)

# 5. PRINT & SAVE
print("\n" + "="*40)
print(f"🏆 RESULTS: DeltaNet ({'Fine-Tuned' if USE_ADAPTER else 'Baseline'})")
print("="*40)
print(make_table(results))

output_file = f"results_deltanet_{'finetuned' if USE_ADAPTER else 'baseline'}.json"
with open(output_file, "w") as f:
    json.dump(results["results"], f, indent=4)
print(f"\n💾 Results saved to {output_file}")

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
HF model type is neither marked as CausalLM or Seq2SeqLM.                     This is expected if your model requires `trust_remote_code=True` but may be an error otherwise.Setting backend to causal
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration



🔌 Plugging DeltaNet into Evaluation Harness...
✅ Force-set Harness Max Length to: 131072
🚀 Starting RULER Evaluation: niah_single_3
📏 Testing Lengths: [2048, 4096, 8192]


niah_single_3: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 2048: 100%|██████████████████████████| 500/500 [00:00<00:00, 520.96it/s]
Generating synthetic samples: essay | 4096: 100%|██████████████████████████| 500/500 [00:02<00:00, 190.08it/s]
Generating synthetic samples: essay | 8192: 100%|███████████████████████████| 500/500 [00:06<00:00, 73.10it/s]
Overwriting default num_fewshot of niah_single_3 from None to 2
Running generate_until requests: 100%|██████████████████████████████████| 1500/1500 [3:06:31<00:00,  7.46s/it]



🏆 RESULTS: DeltaNet (Baseline)
|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_3|      1|none  |     2|  2048|   |0.094|±  |0.0131|
|             |       |none  |     2|  4096|↑  |0.060|±  |   N/A|
|             |       |none  |     2|  8192|↑  |0.022|±  |   N/A|


💾 Results saved to results_deltanet_baseline.json


In [4]:
# ==============================================================================
# ABLATION TEST: Chunked vs Recurrent Mode for DeltaNet
# ==============================================================================
# This tests whether the computation mode affects NIAH accuracy
# Run this BEFORE loading the model (or restart kernel between tests)

import os
import gc
import torch
from lm_eval.models.huggingface import HFLM
from lm_eval import simple_evaluate
from lm_eval.utils import make_table

# --- TEST CONFIGURATION ---
TASK = "niah_single_2"
LENGTHS = [2048]
LIMIT = 20  # Small sample for quick comparison
BASE_MODEL_ID = "fla-hub/delta_net-1.3B-100B"

results_summary = {}

# ==============================================================================
# TEST 1: Default Chunked Mode (Triton)
# ==============================================================================
print("=" * 60)
print("🧪 TEST 1: CHUNKED MODE (Default Triton)")
print("=" * 60)

# Set environment for chunked mode
os.environ["FLA_BACKEND"] = "triton"
if "FLA_CHUNK_SIZE" in os.environ:
    del os.environ["FLA_CHUNK_SIZE"]

# Import FLA after setting env vars
import fla.modules.mlp
from fla.models.delta_net import DeltaNetForCausalLM, DeltaNetConfig
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

# Disable fused SwiGLU to avoid Triton issues
fla.modules.mlp.GatedMLP.fuse_swiglu = False
DeltaNetConfig.fuse_swiglu = False

# Register architecture
try:
    AutoConfig.register("delta_net", DeltaNetConfig)
    AutoModelForCausalLM.register(DeltaNetConfig, DeltaNetForCausalLM)
except ValueError:
    pass

# Load model
model_chunked = DeltaNetForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    trust_remote_code=True,
    device_map="cuda:0",
    torch_dtype=torch.bfloat16,
)
model_chunked.eval()

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Wrap and evaluate
lm_obj = HFLM(pretrained=model_chunked, tokenizer=tokenizer, batch_size=1, trust_remote_code=True)
lm_obj._max_length = 131072

results_chunked = simple_evaluate(
    model=lm_obj,
    tasks=[TASK],
    device="cuda",
    num_fewshot=0,
    limit=LIMIT,
    metadata={"max_seq_lengths": LENGTHS, "tokenizer": BASE_MODEL_ID}
)

print("\n📊 CHUNKED MODE RESULTS:")
print(make_table(results_chunked))
results_summary["chunked"] = results_chunked

# Cleanup
del model_chunked, lm_obj
gc.collect()
torch.cuda.empty_cache()

# ==============================================================================
# TEST 2: Force Small Chunk Size (More Sequential-like)
# ==============================================================================
print("\n" + "=" * 60)
print("🧪 TEST 2: SMALL CHUNK SIZE (chunk_size=64)")
print("=" * 60)

# Force smaller chunk size (more sequential behavior)
os.environ["FLA_CHUNK_SIZE"] = "64"

# Reload model with new settings
# Note: You may need to restart kernel for env vars to fully take effect
model_sequential = DeltaNetForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    trust_remote_code=True,
    device_map="cuda:0",
    torch_dtype=torch.bfloat16,
)
model_sequential.eval()

lm_obj2 = HFLM(pretrained=model_sequential, tokenizer=tokenizer, batch_size=1, trust_remote_code=True)
lm_obj2._max_length = 131072

results_sequential = simple_evaluate(
    model=lm_obj2,
    tasks=[TASK],
    device="cuda",
    num_fewshot=0,
    limit=LIMIT,
    metadata={"max_seq_lengths": LENGTHS, "tokenizer": BASE_MODEL_ID}
)

print("\n📊 SMALL CHUNK RESULTS:")
print(make_table(results_sequential))
results_summary["small_chunk"] = results_sequential

# ==============================================================================
# COMPARISON SUMMARY
# ==============================================================================
print("\n" + "=" * 60)
print("📈 ABLATION SUMMARY")
print("=" * 60)
print(f"Task: {TASK} @ {LENGTHS[0]} tokens")
print(f"Samples: {LIMIT}")
print()
print("Mode           | Accuracy")
print("-" * 30)
for mode, res in results_summary.items():
    # Extract accuracy (adjust key based on actual result structure)
    try:
        acc = res["results"][TASK].get("acc,none", res["results"][TASK].get("exact_match,none", "N/A"))
        print(f"{mode:14} | {acc}")
    except:
        print(f"{mode:14} | See table above")

🧪 TEST 1: CHUNKED MODE (Default Triton)


`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
niah_single_2: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 2048: 100%|████████████████████████████████████| 500/500 [00:00<00:00, 635.96it/s]
Overwriting default num_fewshot of niah_single_2 from None to 0
Running generate_until requests: 100%|██████████████████████████████████████████████████| 20/20 [02:50<00:00,  8.53s/it]



📊 CHUNKED MODE RESULTS:
|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_2|      1|none  |     0|  2048|   |    1|±  |     0|
|             |       |none  |     0|  4096|↑  |   -1|±  |   N/A|


🧪 TEST 2: SMALL CHUNK SIZE (chunk_size=64)


`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
niah_single_2: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 2048: 100%|████████████████████████████████████| 500/500 [00:00<00:00, 635.65it/s]
Overwriting default num_fewshot of niah_single_2 from None to 0
Running generate_until requests: 100%|██████████████████████████████████████████████████| 20/20 [02:40<00:00,  8.05s/it]



📊 SMALL CHUNK RESULTS:
|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_2|      1|none  |     0|  2048|   |    1|±  |     0|
|             |       |none  |     0|  4096|↑  |   -1|±  |   N/A|


📈 ABLATION SUMMARY
Task: niah_single_2 @ 2048 tokens
Samples: 20

Mode           | Accuracy
------------------------------
chunked        | N/A
small_chunk    | N/A
